In [101]:
from hatchet import *
import pandas as pd
import numpy as np
import json
import math
from functools import reduce

In [54]:
class UnionFind(object):
    """Union-find disjoint sets datastructure.

    Union-find is a data structure that maintains disjoint set
    (called connected components or components in short) membership,
    and makes it easier to merge (union) two components, and to find
    if two elements are connected (i.e., belong to the same
    component).

    This implements the "weighted-quick-union-with-path-compression"
    union-find algorithm.  Only works if elements are immutable
    objects.

    Worst case for union and find: :math:`(N + M \log^* N)`, with
    :math:`N` elements and :math:`M` unions. The function
    :math:`\log^*` is the number of times needed to take :math:`\log`
    of a number until reaching 1. In practice, the amortized cost of
    each operation is nearly linear [1]_.

    Terms
    -----
    Component
        Elements belonging to the same disjoint set

    Connected
        Two elements are connected if they belong to the same component.

    Union
        The operation where two components are merged into one.

    Root
        An internal representative of a disjoint set.

    Find
        The operation to find the root of a disjoint set.

    Parameters
    ----------
    elements : NoneType or container, optional, default: None
        The initial list of elements.

    Attributes
    ----------
    n_elts : int
        Number of elements.

    n_comps : int
        Number of distjoint sets or components.

    Implements
    ----------
    __len__
        Calling ``len(uf)`` (where ``uf`` is an instance of ``UnionFind``)
        returns the number of elements.

    __contains__
        For ``uf`` an instance of ``UnionFind`` and ``x`` an immutable object,
        ``x in uf`` returns ``True`` if ``x`` is an element in ``uf``.

    __getitem__
        For ``uf`` an instance of ``UnionFind`` and ``i`` an integer,
        ``res = uf[i]`` returns the element stored in the ``i``-th index.
        If ``i`` is not a valid index an ``IndexError`` is raised.

    __setitem__
        For ``uf`` and instance of ``UnionFind``, ``i`` an integer and ``x``
        an immutable object, ``uf[i] = x`` changes the element stored at the
        ``i``-th index. If ``i`` is not a valid index an ``IndexError`` is
        raised.

    .. [1] http://algs4.cs.princeton.edu/lectures/

    """

    def __init__(self, elements=None):
        self.n_elts = 0  # current num of elements
        self.n_comps = 0  # the number of disjoint sets or components
        self._next = 0  # next available id
        self._elts = []  # the elements
        self._indx = {}  #  dict mapping elt -> index in _elts
        self._par = []  # parent: for the internal tree structure
        self._siz = []  # size of the component - correct only for roots

        if elements is None:
            elements = []
        for elt in elements:
            self.add(elt)

    def get_elts(self):
        return self._elts

    def __repr__(self):
        return  (
            '<UnionFind:\n\telts={},\n\tsiz={},\n\tpar={},\nn_elts={},n_comps={}>'
            .format(
                self._elts,
                self._siz,
                self._par,
                self.n_elts,
                self.n_comps,
            ))

    def __len__(self):
        return self.n_elts

    def __contains__(self, x):
        if x['name'] in self._indx:
            obj_pos = self.find(x)
            obj = self._elts[obj_pos]
            if(x['graphID'] not in obj['graphID']):
                obj['graphID'].append(x['graphID'][0])
            print("Obj", obj)
            
        return x["name"] in self._indx

    def __getitem__(self, index):
        if index < 0 or index >= self._next:
            raise IndexError('index {} is out of bound'.format(index))
        return self._elts[index]

    def __setitem__(self, index, x):
        if index < 0 or index >= self._next:
            raise IndexError('index {} is out of bound'.format(index))
        self._elts[index] = x

    def add(self, x):
        """Add a single disjoint element.

        Parameters
        ----------
        x : immutable object

        Returns
        -------
        None

        """
        if x in self:
            return
        self._elts.append(x)
        self._indx[x["name"]] = self._next
        self._par.append(self._next)
        self._siz.append(1)
        self._next += 1
        self.n_elts += 1
        self.n_comps += 1

    def find(self, x):
        """Find the root of the disjoint set containing the given element.

        Parameters
        ----------
        x : immutable object

        Returns
        -------
        int
            The (index of the) root.

        Raises
        ------
        ValueError
            If the given element is not found.

        """
        if x["name"] not in self._indx:
            raise ValueError('{} is not an element'.format(x))

        p = self._indx[x["name"]]
        while p != self._par[p]:
            # path compression
            q = self._par[p]
            self._par[p] = self._par[q]
            p = q
        return p

    def connected(self, x, y):
        """Return whether the two given elements belong to the same component.

        Parameters
        ----------
        x : immutable object
        y : immutable object

        Returns
        -------
        bool
            True if x and y are connected, false otherwise.

        """
        return self.find(x) == self.find(y)

    def union(self, x, y):
        """Merge the components of the two given elements into one.

        Parameters
        ----------
        x : immutable object
        y : immutable object

        Returns
        -------
        None

        """
        # Initialize if they are not already in the collection
        for elt in [x, y]:
            if elt not in self:
                self.add(elt)

        xroot = self.find(x)
        yroot = self.find(y)
        if xroot == yroot:
            return
        if self._siz[xroot] < self._siz[yroot]:
            self._par[xroot] = yroot
            self._siz[yroot] += self._siz[xroot]
        else:
            self._par[yroot] = xroot
            self._siz[xroot] += self._siz[yroot]
        self.n_comps -= 1

    def component(self, x):
        """Find the connected component containing the given element.

        Parameters
        ----------
        x : immutable object

        Returns
        -------
        set

        Raises
        ------
        ValueError
            If the given element is not found.

        """
        if x not in self:
            raise ValueError('{} is not an element'.format(x))
        elts = np.array(self._elts)
        vfind = np.vectorize(self.find)
        roots = vfind(elts)
        return set(elts[roots == self.find(x)])

    def components(self):
        """Return the list of connected components.

        Returns
        -------
        list
            A list of sets.

        """
        elts = np.array(self._elts)
        vfind = np.vectorize(self.find)
        roots = vfind(elts)
        distinct_roots = set(roots)
        return [set(elts[roots == root]) for root in distinct_roots]
        # comps = []
        # for root in distinct_roots:
        #     mask = (roots == root)
        #     comp = set(elts[mask])
        #     comps.append(comp)
        # return comps

    def component_mapping(self):
        """Return a dict mapping elements to their components.

        The returned dict has the following semantics:

            `elt -> component containing elt`

        If x, y belong to the same component, the comp(x) and comp(y)
        are the same objects (i.e., share the same reference). Changing
        comp(x) will reflect in comp(y).  This is done to reduce
        memory.

        But this behaviour should not be relied on.  There may be
        inconsitency arising from such assumptions or lack thereof.

        If you want to do any operation on these sets, use caution.
        For example, instead of

        ::

            s = uf.component_mapping()[item]
            s.add(stuff)
            # This will have side effect in other sets

        do

        ::

            s = set(uf.component_mapping()[item]) # or
            s = uf.component_mapping()[item].copy()
            s.add(stuff)

        or

        ::

            s = uf.component_mapping()[item]
            s = s | {stuff}  # Now s is different

        Returns
        -------
        dict
            A dict with the semantics: `elt -> component contianing elt`.

        """
        elts = np.array(self._elts)
        vfind = np.vectorize(self.find)
        roots = vfind(elts)
        distinct_roots = set(roots)
        comps = {}
        for root in distinct_roots:    
            mask = [(r == root) for r in roots] 
            comp = elts[mask]
            comps.update({x["name"]: comp for x in comp})
            # Change ^this^, if you want a different behaviour:
            # If you don't want to share the same set to different keys:
            # comps.update({x: set(comp) for x in comp})
        return comps


In [63]:
path = "/home/vidi/Work/llnl/CallFlow/data/kripke/"
datasets = ['impi', 'mvapich2']

graph_files = {}
df_files = {}
graph_file = 'sample.json'
for dataset in datasets:
    graph_files[dataset] = path + dataset + '/' + graph_file
    
df_file = 'df.csv'
for dataset in datasets:
    df_files[dataset] = path + dataset + '/' + df_file
    
gfs = {}
for idx, dataset in enumerate(graph_files):
    with open(graph_files[dataset], 'r') as graphFile:
        data = json.load(graphFile)
        gf = GraphFrame()
        gf.from_literal(data)
        gfs[dataset] = gf
        gfs[dataset].df = pd.read_csv(df_files[dataset])
print(gfs)

<program root>
main
main
<program root>
main
main
main
{'impi': <hatchet.graphframe.GraphFrame object at 0x7f406ebbae48>, 'mvapich2': <hatchet.graphframe.GraphFrame object at 0x7f406ebb6908>}


In [158]:
def dfs(df, graph, graph_name):        
        def dfs_recurse(root):
            for node in root.children:
                source = root
                target = node
                source_name = source.callpath[-1]
                target_name = target.callpath[-1]
                print("Node : {0} - {1}".format(source_name, target_name))
                union_find.add({
                    "name": source_name + '-' + target_name,
                    "source": source,
                    "target": target,
                    "graphID": [graph_name],
                    "source_time": df.loc[df['name'] == source_name]['time'].mean(),
                    "target_time": df.loc[df['name'] == target_name]['time'].mean(),
               })
                dfs_recurse(node)
        for root in graph.roots:
            print("Node: {0}".format(root.callpath[-1]))
            dfs_recurse(root)

In [182]:
graph_impi = gfs['impi'].graph
graph_mvapich2 = gfs['mvapich2'].graph
df_impi = gfs['impi'].df
df_mvapich2 = gfs['mvapich2'].df

union_find = UnionFind()

dfs(df_impi, graph_impi, "impi")
dfs(df_mvapich2, graph_mvapich2, "mvapich2")
edges = union_find.get_elts()

nodes = []
for edge in edges:
    source_node = edge["source"]
    target_node = edge["target"]
    source = {}
    target = {}
    if(source not in nodes):
        source["name"] = edge["source"].callpath[-1]
        source["diff_time_avg"] = findAvgDiff(gfs, source_node, 'time')
        source["diff_time_avg (inc)"] = findAvgDiff(gfs, source_node, 'time (inc)')
        source["diff_time_min"] = findMinDiff(gfs, source_node, 'time')
        source["diff_time_min (inc)"] = findMinDiff(gfs, source_node, 'time (inc)')
        source["imbalance_perc"] = findAvgDiff(gfs, source_node, "imbalance_perc")
        nodes.append(source)
        
    if(target not in nodes):
        target["name"] = edge["target"].callpath[-1]
        target["diff_time_avg"] = findAvgDiff(gfs, target_node, 'time')
        target["diff_time_avg (inc)"] = findAvgDiff(gfs, target_node, 'time (inc)')
        target["diff_time_min"] = findMinDiff(gfs, target_node, 'time')
        target["diff_time_min (inc)"] = findMinDiff(gfs, target_node, 'time (inc)')
        target["imbalance_perc"] = findAvgDiff(gfs, target_node, "imbalance_perc")
        nodes.append(target)
    
for node in nodes:
    print(node)

Node: <program root>
Node : <program root> - main
Node : main - func2
Node : main - func1
Node: <program root>
Node : <program root> - main
Obj {'name': '<program root>-main', 'source': <hatchet.node.Node object at 0x7f406ec25e80>, 'target': <hatchet.node.Node object at 0x7f406ec25400>, 'graphID': ['impi', 'mvapich2'], 'source_time': 0.0, 'target_time': 0.0}
Node : main - func1
Obj {'name': 'main-func1', 'source': <hatchet.node.Node object at 0x7f406ec25400>, 'target': <hatchet.node.Node object at 0x7f406ec25d68>, 'graphID': ['impi', 'mvapich2'], 'source_time': 0.0, 'target_time': nan}
Node : main - func2
Obj {'name': 'main-func2', 'source': <hatchet.node.Node object at 0x7f406ec25400>, 'target': <hatchet.node.Node object at 0x7f406ec252b0>, 'graphID': ['impi', 'mvapich2'], 'source_time': 0.0, 'target_time': nan}
Node : main - func3
<program root>
[0.0, 0.0]
0.0
[0.0, 0.0]
<program root>
[194682761.203125, 194516926.140625]
389199687.34375
[-194516926.140625, -194682761.203125]
0.0
[0.

In [179]:
target["diff_time (inc)"] = findAvgDiff(gfs, edge['source'], 'time (inc)')

main
[194682761.203125, 194516926.140625]
389199687.34375
[-194516926.140625, -194682761.203125]


In [181]:
def findAvgDiff(gfs, node, attr):
    attr_values = []
    for gf_name in gfs.keys():
        df = gfs[gf_name].df
        attr_value = df.loc[df['name'] == node.callpath[-1]][attr].mean()
        attr_values.append(attr_value)

    sum_attr_values = 0
    avg_attr_values = reduce(lambda sum_attr_values, x: (sum_attr_values + x), attr_values)
        
    diff_by_avg = []
    for idx, avg in enumerate(attr_values):
        diff_by_avg.append(avg - avg_attr_values)
    
    return diff_by_avg

def findMinDiff(gfs, node, attr):    
    attr_values = []
    for gf_name in gfs.keys():
        df = gfs[gf_name].df
        attr_value = df.loc[df['name'] == node.callpath[-1]][attr].mean()
        attr_values.append(attr_value)

    min_ = attr_values[0]
    for idx, attr_value in enumerate(attr_values):
        min_ = min(min_, attr_value)

    diff_by_min = []
    for idx, attr_value in enumerate(attr_values):
        diff_by_min.append(attr_value - min_)
    
    return diff_by_min

In [69]:
def trees_to_literal(graph, dataframe):
    """ Calls to_json in turn for each tree in the graph/forest
    """
    literal = []
    nodes = dataframe['name'].unique()
    adj_idx_map = {}
    for idx, node in enumerate(nodes):
        adj_idx_map[node] = idx

    num_of_nodes = len(nodes)
    adj_matrix = np.zeros(shape=(num_of_nodes, num_of_nodes))

    mapper = {}

    def add_nodes_and_children(hnode):
        node_name = dataframe.loc[dataframe['node'] == hnode]['name'][0]
        children = []
        # print(node_name)
        # if(node_name in mapper):
        #     mapper[node_name] = 0
        # else:
        #     mapper[node_name] = 1
        
        # print(mapper[node_name])

        # There is some bug somewhere. 
        for child in hnode.children:
            child_name = dataframe.loc[dataframe['node'] == child]['name'][0]
            if child_name in adj_idx_map and node_name in adj_idx_map:
                source_idx = adj_idx_map[node_name]
                target_idx = adj_idx_map[child_name]
                if(adj_matrix[source_idx][target_idx] == 0.0):
                    adj_matrix[source_idx, target_idx] = 1.0                
            # if(mapper[node_name] == 1):
            children.append(add_nodes_and_children(child))

        return {
            "name": node_name,
            "children": children,
            "metrics": {}
        }

    for root in graph.roots:
        literal.append(add_nodes_and_children(root))

    return literal